In [8]:
import sys, os
from pathlib import Path
from tops.config import LazyConfig
sys.path.append(os.path.dirname(os.getcwd())) # Include ../SSD in path

In [3]:
print(os.path.dirname(os.getcwd()))

/lhome/ivang/tdt4265/TDT4265_Assignment1/assignment4/SSD


# Introduction to SSD

This code is much more complex than your prior assignment, and we recommend you to spend some time getting familiar with the code structure.
The "complex" code structure is made to simplify aspects of deep learning experimentation, and help you structure the usual "sphagetti" deep learning code.


All scripts in this code requires a configuration file. To **start training**, you can type:
```
python train.py configs/ssd300.py
```

## Configuration files
The key difference from previous starter codes is the use of configuration files. This enables us to change small parts of the experiment without changing hard-coded values (e.g. learning rate in previous assignments).

If you take a look in [`configs/ssd300.py`](../configs/ssd300.py) you will notice a large set of objects describing model architecture (`backbone` and `model`), the optimizer, dataset, data loading, and hyperparameters.

To load the config we can write the following:

In [9]:
from ssd.utils import load_config
#cfg = load_config("../configs/ssd300.py")
path = "../configs/ssd300.py"

config_path = Path(path)
run_name = "_".join(config_path.parts[1:-1]) + "_" + config_path.stem
cfg = LazyConfig.load(os.fspath(config_path))
cfg.output_dir = Path(cfg.train._output_dir).joinpath(*config_path.parts[1:-1], config_path.stem)
cfg.run_name = run_name

Saving SSD outputs to: outputs/


`cfg` supports access syntax, where all objects in `configs/ssd300.py` are accessible via their attribute name.



In [10]:
print(cfg.model)

{'feature_extractor': '${backbone}', 'anchors': '${anchors}', 'loss_objective': '${loss_objective}', 'num_classes': 11, '_target_': <class 'ssd.modeling.ssd.SSD300'>}


If we print `cfg.model`, notice that it returns a dictionary and not the model object itself (which is `SSD300` from [`ssd/modeling/ssd.py`](../ssd/modeling/ssd.py)). This is because the model is defined "lazily" (wrapped with a `LazyCall`).

To create the model, we have to instantiate it:

In [12]:
from tops.config import instantiate
model = instantiate(cfg.model)
print(model)

SSD300(
  (feature_extractor): BasicModel(
    (features): ModuleList(
      (0): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): ReLU()
        (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (7): ReLU()
        (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (9): ReLU()
      )
      (1): Sequential(
        (0): ReLU()
        (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (2): ReLU()
        (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (4): ReLU()
      )
      (2): Sequential(
        (0): ReLU()
        (1): Conv2d(256, 256, 

Another example, we can load the first batch of the dataset and run a forward pass with the model:

In [13]:
dataloader_train = instantiate(cfg.data_train.dataloader)
print(type(dataloader_train))
batch = next(iter(dataloader_train))
for key, item in batch.items():
    print(key, "has the shape:", item.shape)
gpu_transform = instantiate(cfg.data_train.gpu_transform)
batch = gpu_transform(batch)
bbox_delta, confidences = model(batch["image"])
print(f"The model predicted anchors with  bbox delta: {bbox_delta.shape} and confidences: {confidences.shape}")

<class 'torch.utils.data.dataloader.DataLoader'>
image has the shape: torch.Size([32, 3, 300, 300])
boxes has the shape: torch.Size([32, 8732, 4])
labels has the shape: torch.Size([32, 8732])
The model predicted anchors with  bbox delta: torch.Size([32, 4, 8732]) and confidences: torch.Size([32, 11, 8732])


You might ask yourself, why? At first sight, this seems very complicated rather than plain-old  hard coded values.

The reason is easy manipulation of experiments. If I want to run the same experiment, but with a different batch size, I can change it with the following:

In [15]:
# Lets print the batch size of the original data loader:
print("Original batch size:", dataloader_train.batch_size)
cfg.train.batch_size = 2 # Setting the batch size to 2
dataloader_train = instantiate(cfg.data_train.dataloader)
print("New batch size:", dataloader_train.batch_size)

Original batch size: 2
New batch size: 32


Another reason is **configuration inheritance**. E.g. for the last task, you are going to train VGG on the VOC dataset. 
This requires us to change the backbone and dataset, however, keep all other parameters the same (e.g. general configs (cfg.train), anchors, schedulers, optimizers, etc.).

Take a look in [`configs/voc_vgg.py`](../configs/voc_vgg.py) and notice that we inherit from the original config:
```
from .ssd300 import train, anchors, optimizer, schedulers, model, data_train, data_val
```
The only changes done are to the backbone, dataset and dataset transforms.

In [21]:
path = "../configs/voc_vgg.py"

config_path = Path(path)
run_name = "_".join(config_path.parts[1:-1]) + "_" + config_path.stem
cfg = LazyConfig.load(os.fspath(config_path))
cfg.output_dir = Path(cfg.train._output_dir).joinpath(*config_path.parts[1:-1], config_path.stem)
cfg.run_name = run_name
print(cfg.schedulers.linear.total_iters)
cfg.schedulers.linear.total_iters = 5000
print(cfg.schedulers.linear.total_iters)
model = instantiate(cfg.model)
print(model)

Saving SSD outputs to: outputs/
Found dataset directory in: /work/datasets/VOCdevkit/VOC2007
Found dataset directory in: /work/datasets/VOCdevkit/VOC2012
Found dataset directory in: /work/datasets/VOCdevkit/VOC2007
500
5000


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SSD300(
  (feature_extractor): VGG(
    (vgg): ModuleList(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      

In [22]:
!python ../train.py ../configs/voc_vgg.py

2023-03-24 19:12:33.427085: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Saving SSD outputs to: outputs/
Found dataset directory in: /work/datasets/VOCdevkit/VOC2007
Found dataset directory in: /work/datasets/VOCdevkit/VOC2012
Found dataset directory in: /work/datasets/VOCdevkit/VOC2007
--------------------Config file below--------------------
{ 'anchors': { '_target_': <class 'ssd.modeling.anchor_boxes.AnchorBoxes'>,
               'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
               'feature_sizes': [ [38, 38],
                                  [19, 19],
                                  [10, 10],
                                  [5, 5],
                                  [3, 3],
                                  [1, 1]],
               'image_shape': '${train.imshape}',
               'min_sizes': [ [15, 15],
                              [30, 30],
                              [111, 111],
 

# Useful commands:


#### Training and evaluation
To start training:
```
python train.py  configs/ssd300.py
```

To starting training VGG on VOC:
```
python train.py  configs/voc_vgg.py
```

To only run evaluation:
```
python train.py  configs/ssd300.py --evaluate-only
```

#### Demo.py
For VOC:
```
python demo.py configs/voc_vgg.py demo/voc demo/voc_output
```

For MNIST:
```
python demo.py configs/ssd300.py demo/mnist demo/mnist_output
```


#### Runtime analysis:
```
python3 runtime_analysis.py configs/ssd300.py
```